# Kogi Notebook - Preparation

## Paths Configuration and Imports

In [ ]:
# Add your Implementation directory to the path
# Adjust this path to match your project structure


import os
import sys
project_root = os.path.abspath(".")  # Current directory
implementation_path = os.path.join(project_root, "Implementation")
app_path = os.path.join(project_root, "App")
new_semantics_path = os.path.join(project_root, "NewSemantics") 

# Add paths to sys.path so we can import your modules
if implementation_path not in sys.path:
    sys.path.append(implementation_path)
if app_path not in sys.path:
    sys.path.append(app_path)
if new_semantics_path not in sys.path:
    sys.path.append(new_semantics_path)

print(f"Project root: {project_root}")
print(f"Implementation path: {implementation_path}")
print(f"App path: {app_path}")
print("Paths configured!")

In [ ]:
# Interactive Goal Model Evaluation - Jupyter Notebook
# Run each cell sequentially to set up the interactive environment

import os
import sys
import json
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import FancyBboxPatch
import networkx as nx
import numpy as np
from enum import Enum
from typing import Dict, List, Tuple, Set
import pandas as pd
from Implementation.enums import ElementStatus, QualityStatus, LinkType, LinkStatus
# from Implementation.goal_model import GoalModel # Old semantics
from NewSemantics.goal_model import GoalModel # New Semantics from the paper
from NewSemantics.istar_processor import read_istar_model
from Ui.Layout import Layout

## Reading the inputs

In [ ]:
# Transfor the goal model .txt file into a model instance
def create_model_from_your_code():
    filepath = "Data/example_from_paper.txt"
    print(f"✓ Model created using istar file {filepath}!")
    print("Press on 'reset model' to replace the previous model")
    return read_istar_model(filepath)

# Create model instance
model = create_model_from_your_code()

## Visualization Functions

In [ ]:
# Cell 5: Visualization Functions
# --- IGNORE --- Warnings of "-----" is not defined (they are the name of the elemnts of the goal model)

def get_status_color_from_your_model(element_id):
    """Get color based on element status using your model's data structures"""
    if element_id in model.qualities:
        status = model.qualities[element_id]
        if status == QualityStatus.UNKNOWN:
            return 'white'
        elif status == QualityStatus.FULFILLED:
            return 'lightgreen'
        elif status == QualityStatus.DENIED:
            return 'lightcoral'
    else:
        status = model.tasks.get(element_id) or model.goals.get(element_id)
        if status == ElementStatus.UNKNOWN:
            return 'white'
        elif status == ElementStatus.TRUE_FALSE:
            return 'lightgreen'
        elif status == ElementStatus.TRUE_TRUE:
            return 'lightblue'
    return 'white'

def create_goal_model_visualization():
    """Create visualization using your actual GoalModel data"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
    
    # Left side: Goal Model Structure
    ax1.set_title("Goal Model Structure", fontsize=14, fontweight='bold')
    ax1.set_xlim(0, 10)
    ax1.set_ylim(0, 10)
    ax1.set_aspect('equal')
    
    # Define positions for elements
    positions = model.positions
    
    # Draw elements using your model's actual data
    for element_id, (x, y) in positions.items():
        color = get_status_color_from_your_model(element_id)
        
        if element_id.startswith('Q'):
            # Quality - cloud shape
            cloud = FancyBboxPatch((x-0.4, y-0.3), 0.8, 0.6, 
                                 boxstyle="round,pad=0.1", 
                                 facecolor=color, edgecolor='black', linewidth=2)
            ax1.add_patch(cloud)
            # Show actual status from your model
            status_text = f"{element_id}\n{model._format_status(model.qualities[element_id])}"
            ax1.text(x, y, status_text, ha='center', va='center', fontweight='bold', fontsize=8)
        elif element_id.startswith('G'):
            # Goal - ellipse
            ellipse = patches.Ellipse((x, y), 0.8, 0.5, 
                                    facecolor=color, edgecolor='black', linewidth=2)
            ax1.add_patch(ellipse)
            status_text = f"{element_id}\n{model._format_status(model.goals[element_id])}"
            ax1.text(x, y, status_text, ha='center', va='center', fontweight='bold', fontsize=8)
        else:
            # Task - hexagon
            hexagon = patches.RegularPolygon((x, y), 6, radius=0.4, 
                                          facecolor=color, edgecolor='black', linewidth=2)
            ax1.add_patch(hexagon)
            status_text = f"{element_id}\n{model._format_status(model.tasks[element_id])}"
            ax1.text(x, y, status_text, ha='center', va='center', fontweight='bold', fontsize=8)
    
    # Draw links using your model's actual links
    for parent, child, link_type, status in model.links:
        if parent in positions and child in positions:
            px, py = positions[parent]
            cx, cy = positions[child]
            
            # Determine arrow style based on link type from your enums
            if link_type == LinkType.MAKE:
                color = 'green'
                style = '->'
            elif link_type == LinkType.BREAK:
                color = 'red'
                style = '->'
            elif link_type == LinkType.AND:
                        arrow_color = 'purple'
                        style = '->'
            elif link_type == LinkType.OR:
                        arrow_color = 'orange'
                        style = '->'
            else:
                color = 'blue'
                style = '->'
            
            ax1.annotate('', xy=(cx, cy), xytext=(px, py),
                        arrowprops=dict(arrowstyle=style, color=color, lw=2))
    
    ax1.set_xticks([])
    ax1.set_yticks([])
    
    # Right side: Centered Petri Net
    ax2.set_title("Process Model (Petri Net)", fontsize=14, fontweight='bold')
    ax2.set_xlim(0, 12)
    ax2.set_ylim(0, 6)
    ax2.set_aspect('equal')
    
    # Define Petri net structure - centered
    petri_elements = {
        # Places (circles) - (x, y, label)
        'places': [
            (0.5, 3, 'p0'),  # Start
            (2, 4, 'p1'), (2, 2, 'p2'),  # After t1
            (3.5, 4, 'p3'), (3.5, 2, 'p4'),  # After t2, t3
            (5.5, 3, 'p5'),  # After t4
            (7, 3, 'p6'),  # After t5, t6
            (8.5, 4, 'p7'), (8.5, 2, 'p9'),  # After t7
            (10, 4, 'p8'), (10, 2.5, 'p9_alt'),  # After t8, t11
            (11.5, 3, 'p10')  # End
        ],
        # Transitions (squares) - (x, y, label)
        'transitions': [
            (1.2, 3, 't1'),  # Split
            (2.75, 4.5, 't2'), (2.75, 1.5, 't3'),  # Parallel
            (4.5, 3, 't4'),  # Join
            (6.25, 3.5, 't5'), (6.25, 2.5, 't6'),  # Parallel
            (7.75, 3, 't7'),  # Join
            (9.25, 4.5, 't8'), (9.25, 1.5, 't11'),  # Parallel
            (9.25, 2.5, 't10'),  # Middle
            (10.75, 3, 't9')  # Final join
        ]
    }
    
    # Draw places (circles)
    for x, y, label in petri_elements['places']:
        if label == 'p0':  # Start place with token
            circle = patches.Circle((x, y), 0.15, facecolor='black', edgecolor='black')
            ax2.add_patch(circle)
            # Add token
            token = patches.Circle((x, y), 0.05, facecolor='white', edgecolor='white')
            ax2.add_patch(token)
        elif label == 'p10':  # End place
            circle = patches.Circle((x, y), 0.15, facecolor='white', edgecolor='black', linewidth=3)
            ax2.add_patch(circle)
        else:
            circle = patches.Circle((x, y), 0.15, facecolor='white', edgecolor='black', linewidth=2)
            ax2.add_patch(circle)
        
        ax2.text(x, y-0.3, label, ha='center', va='top', fontsize=8)
    
    # Draw transitions (squares)
    executed_events = getattr(create_trace_visualization, 'executed_events', [])
    for x, y, label in petri_elements['transitions']:
        # Color based on execution status
        if label.replace('t', 'e') in executed_events:
            color = 'lightgreen'
        else:
            color = 'black'
        
        square = patches.Rectangle((x-0.1, y-0.1), 0.2, 0.2, 
                                 facecolor=color, edgecolor='black', linewidth=2)
        ax2.add_patch(square)
        ax2.text(x, y-0.35, label, ha='center', va='top', fontsize=8, fontweight='bold')
    
    # Define arcs (connections)
    arcs = [
        # From p0 to t1
        ((0.5, 3), (1.2, 3)),
        # From t1 to p1 and p2
        ((1.2, 3), (2, 4)), ((1.2, 3), (2, 2)),
        # From p1 to t2, from p2 to t3
        ((2, 4), (2.75, 4.5)), ((2, 2), (2.75, 1.5)),
        # From t2 to p3, from t3 to p4
        ((2.75, 4.5), (3.5, 4)), ((2.75, 1.5), (3.5, 2)),
        # From p3 and p4 to t4
        ((3.5, 4), (4.5, 3)), ((3.5, 2), (4.5, 3)),
        # From t4 to p5
        ((4.5, 3), (5.5, 3)),
        # From p5 to t5 and t6
        ((5.5, 3), (6.25, 3.5)), ((5.5, 3), (6.25, 2.5)),
        # From t5 and t6 to p6
        ((6.25, 3.5), (7, 3)), ((6.25, 2.5), (7, 3)),
        # From p6 to t7
        ((7, 3), (7.75, 3)),
        # From t7 to p7 and p9
        ((7.75, 3), (8.5, 4)), ((7.75, 3), (8.5, 2)),
        # From p7 to t8, from p9 to t11 and t10
        ((8.5, 4), (9.25, 4.5)), ((8.5, 2), (9.25, 1.5)), ((8.5, 2), (9.25, 2.5)),
        # From t8 to p8, from t11 to p9_alt
        ((9.25, 4.5), (10, 4)), ((9.25, 1.5), (10, 2.5)),
        # From p8, p9_alt and t10 to t9
        ((10, 4), (10.75, 3)), ((10, 2.5), (10.75, 3)), ((9.25, 2.5), (10.75, 3)),
        # From t9 to p10
        ((10.75, 3), (11.5, 3))
    ]
    
    # Draw arcs
    for (x1, y1), (x2, y2) in arcs:
        ax2.annotate('', xy=(x2, y2), xytext=(x1, y1),
                    arrowprops=dict(arrowstyle='->', color='black', lw=1.5))
    
    ax2.set_xticks([])
    ax2.set_yticks([])
    
    plt.tight_layout()
    return fig

print("Visualization functions ready!")

## Interactive Controls

In [ ]:
# Cell 6: Interactive Controls 

def create_interactive_controls():
    """Create interactive controls that use your GoalModel's process_event method"""
    
    # Process transition dropdown using your actual event mappings
    process_dropdown = widgets.Dropdown(
        options=list(model.event_mapping.keys()),
        value=list(model.event_mapping.keys())[0],
        description='Event:',
        disabled=False,
    )
    
    # Execute button
    execute_button = widgets.Button(
        description='Execute Event',
        disabled=False,
        button_style='success',
        tooltip='Execute the selected event using your process_event method'
    )
    
    # Reset button
    reset_button = widgets.Button(
        description='Reset Model',
        disabled=False,
        button_style='warning',
        tooltip='Reset the model to initial state'
    )
    
    # Output areas
    trace_output = widgets.Output()
    status_output = widgets.Output()
    viz_output = widgets.Output()
    
    def execute_event(b):
        """Execute selected event using your GoalModel's process_event method"""
        with status_output:
            clear_output(wait=True)
            selected_event = process_dropdown.value
            
            print(f"Executing event: {selected_event}")
            
            # Use your actual process_event method
            model.process_event(selected_event)
            
            print(f"Event {selected_event} processed successfully")
            
            # Show current quality status using your model's data
            print("\nCurrent Status:")
            for quality_id, status in model.qualities.items():
                formatted_status = model._format_status(status)
                print(f"  {quality_id}: {formatted_status}")
        
        update_trace_display()
        update_visualization()
    
    def reset_model(b):
        """Reset the model using your create_model logic"""
        global model
        model = create_model_from_your_code()
        
        with status_output:
            clear_output(wait=True)
            print("Model reset to initial state")
        
        with trace_output:
            clear_output(wait=True)
            print("Event sequence: []")
        
        update_visualization()
    
    def update_trace_display():
        """Update the trace display"""
        with trace_output:
            clear_output(wait=True)
            # You can access execution history through your model if available
            # For now, we'll show the current state
            print("Current Model State:")
            print(f"Execution counts: {model.execution_count}")
    
    def update_visualization():
        """Update the visualization using your model data"""
        with viz_output:
            clear_output(wait=True)
            fig = create_goal_model_visualization()
            plt.show()
            plt.close()
    
    # Connect event handlers
    execute_button.on_click(execute_event)
    reset_button.on_click(reset_model)
    
    # Create layout
    controls = widgets.HBox([process_dropdown, execute_button, reset_button])
    
    # Initial displays
    update_trace_display()
    update_visualization()
    
    return controls, trace_output, status_output, viz_output

# Create and display controls
controls, trace_out, status_out, viz_out = create_interactive_controls()
print("Interactive controls created using your GoalModel!")

## Trace and Evolution (compare one by one) of both systems

In [ ]:
# Cell 7: Trace and Evolution Visualization Functions

def create_trace_visualization():
    """Create horizontal trace timeline showing only events"""
    trace_output = widgets.Output()
    
    # Global variable to track executed events
    if not hasattr(create_trace_visualization, 'executed_events'):
        create_trace_visualization.executed_events = []
    
    def update_trace_display():
        with trace_output:
            clear_output(wait=True)
            
            # Create horizontal trace timeline
            trace_html = """
            <div style='border: 2px solid #ccc; padding: 15px; margin: 10px; background-color: #f9f9f9;'>
                <h3>Trace Execution Timeline</h3>
            """
            
            if not create_trace_visualization.executed_events:
                trace_html += "<p>No events executed yet. Select an event and click 'Execute Event' to start.</p>"
            else:
                # Create horizontal event sequence
                trace_html += "<div style='display: flex; align-items: center; gap: 10px; font-size: 18px; font-weight: bold;'>"
                trace_html += "<span style='color: #666;'>trace ⟨</span>"
                
                for i, event in enumerate(create_trace_visualization.executed_events):
                    if i > 0:
                        trace_html += "<span style='color: #666;'>,</span>"
                    trace_html += f"<span style='color: #2E86AB; margin: 0 5px;'>{event}</span>"
                
                trace_html += "<span style='color: #666;'>⟩</span>"
                trace_html += "</div>"
            
            trace_html += "</div>"
            display(HTML(trace_html))
    
    return trace_output, update_trace_display


def create_evolution_visualization():
    """Create evolution view with proper mapping visualization"""
    evolution_out = widgets.Output()
    
    def update_evolution():
        with evolution_out:
            clear_output(wait=True)
            
            # Get executed events
            executed_events = getattr(create_trace_visualization, 'executed_events', [])
            
            # Build mapping from events to elements
            event_to_element_mapping = {}
            
            # Try to get mappings from your model
            try:
                if hasattr(model, 'events') and hasattr(model, 'get_mapping'):
                    for event in model.events:
                        mapped_element = model.get_mapping(event)
                        if mapped_element:
                            event_to_element_mapping[event] = mapped_element
                else:
                    # Fallback: define mappings based on your model structure
                    event_to_element_mapping = {
                        'e1': 'T1', 'e2': 'T2', 'e3': 'T3', 'e4': 'T4',
                        'e5': 'T5', 'e6': 'T6', 'e7': 'T7', 'e8': 'T8'
                    }
            except:
                # Default mappings if model access fails
                event_to_element_mapping = {
                    'e1': 'T1', 'e2': 'T2', 'e3': 'T3', 'e4': 'T4',
                    'e5': 'T5', 'e6': 'T6', 'e7': 'T7', 'e8': 'T8'
                }
            
            # Get which elements should be highlighted
            highlighted_elements = set()
            for executed_event in executed_events:
                if executed_event in event_to_element_mapping:
                    highlighted_elements.add(event_to_element_mapping[executed_event])
            
            # Display the evolution view
            display(HTML(f"""
            <div style='background-color: #2d2d2d; padding: 20px; border-radius: 8px; font-family: "Segoe UI", Arial, sans-serif;'>
                <div style='text-align: center; margin-bottom: 20px;'>
                    <div style='background-color: #404040; color: #ffffff; padding: 10px; border-radius: 4px; display: inline-block; min-width: 100px;'>
                        {executed_events[-1] if executed_events else 'Initial State'}
                    </div>
                    <div style='color: #b0b0b0; margin-top: 8px; font-size: 12px;'>Current Event</div>
                </div>
                
                <div style='display: flex; justify-content: space-between; max-width: 400px; margin: 0 auto;'>
                    <div style='text-align: center;'>
                        <div style='color: #ffffff; font-weight: 600; margin-bottom: 15px;'>Process</div>
                        {''.join([
                            f'<div style="background-color: {"#28a745" if event in executed_events else "#404040"}; '
                            f'color: #ffffff; padding: 8px 12px; margin: 5px 0; border-radius: 4px; '
                            f'border: {"2px solid #ffffff" if event == (executed_events[-1] if executed_events else None) else "1px solid #666"};">'
                            f'{event}</div>'
                            for event in ['e1', 'e2', 'e3', 'e4', 'e5', 'e6', 'e7', 'e8']
                        ])}
                    </div>
                    
                    <div style='display: flex; align-items: center; margin: 0 20px;'>
                        <div style='color: #4a90e2; font-size: 24px;'>→</div>
                    </div>
                    
                    <div style='text-align: center;'>
                        <div style='color: #ffffff; font-weight: 600; margin-bottom: 15px;'>Elements</div>
                        {''.join([
                            f'<div style="background-color: {"#28a745" if element in highlighted_elements else "#6c757d"}; '
                            f'color: #ffffff; padding: 8px 12px; margin: 5px 0; border-radius: 4px; '
                            f'border: 1px solid #666;">{element}</div>'
                            for element in ['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8']
                        ])}
                        {''.join([
                            f'<div style="background-color: #6c757d; color: #ffffff; padding: 8px 12px; '
                            f'margin: 5px 0; border-radius: 50%; border: 1px solid #666;">{goal}</div>'
                            for goal in ['G1', 'G2', 'G3', 'Q1']
                        ])}
                    </div>
                </div>
                
                <div style='text-align: center; margin-top: 20px; color: #b0b0b0; font-size: 12px;'>
                    {'Mapping: ' + ', '.join([f"{k} → {v}" for k, v in event_to_element_mapping.items() if k in executed_events]) if executed_events else 'No mappings executed'}
                </div>
            </div>
            """))
    
    return evolution_out, update_evolution


def create_dual_model_visualization():
    """Create vertically stacked models with horizontal mapping table at bottom"""
    viz_output = widgets.Output()
    
    def update_dual_visualization():
        with viz_output:
            clear_output(wait=True)
            
            # Create figure with 3 subplots: petri net top, goal model middle, mappings bottom
            fig = plt.figure(figsize=(20, 16))
            
            # Create grid layout: 3 rows, models take full width, mappings at bottom
            gs = fig.add_gridspec(3, 1, height_ratios=[1.2, 1.2, 0.4], hspace=0.35)
            ax1 = fig.add_subplot(gs[0, 0])  # Petri net (top)
            ax2 = fig.add_subplot(gs[1, 0])  # Goal model (middle)
            ax3 = fig.add_subplot(gs[2, 0])  # Mappings table (bottom, horizontal)
            
            # Top: Petri Net (full width) - matching the LaTeX figure
            ax1.set_title("Process Model (Petri Net)", fontsize=16, fontweight='bold', pad=20)
            ax1.set_xlim(-1, 19)
            ax1.set_ylim(-1, 8)
            ax1.set_aspect('equal')
            
            # Recreate the exact Petri net structure from the LaTeX figure
            petri_elements = {
                # Places (circles) - (x, y, label) - based on LaTeX coordinates
                'places': [
                    (0.8, 3.5, 'p0'),      # Start place with token
                    (3.2, 5.5, 'p1'),      # Top branch after t1
                    (3.2, 1.5, 'p2'),      # Bottom branch after t1
                    (6.0, 5.5, 'p3'),      # After t2
                    (6.0, 1.5, 'p4'),      # After t3
                    (8.5, 3.5, 'p5'),      # After t4 (merge)
                    (11.5, 3.5, 'p6'),     # After t5/t6
                    (14.0, 5.5, 'p7'),     # After t7 (top)
                    (14.0, 1.5, 'p9'),     # After t7 (bottom)
                    (16.5, 5.5, 'p8'),     # After t8
                    (17.5, 3.5, 'p10')     # End place
                ],
                # Transitions (squares) - (x, y, label)
                'transitions': [
                    (1.8, 3.5, 't_1'),     # Initial split
                    (4.6, 5.5, 't_2'),     # Top parallel
                    (4.6, 1.5, 't_3'),     # Bottom parallel
                    (7.25, 3.5, 't_4'),    # Merge
                    (10.0, 4.2, 't_5'),    # Split again (top)
                    (10.0, 2.8, 't_6'),    # Split again (bottom)
                    (12.75, 3.5, 't_7'),   # Another merge
                    (15.25, 5.5, 't_8'),   # Top final
                    (16.0, 3.5, 't_9'),    # Final merge
                    (16.75, 2.5, 't_10'),  # Middle
                    (14.75, 1.5, 't_11')   # Bottom final
                ]
            }
            
            # Draw places (circles)
            for x, y, label in petri_elements['places']:
                if label == 'p0':  # Start place with token
                    circle = patches.Circle((x, y), 0.25, facecolor='white', edgecolor='black', linewidth=2)
                    ax1.add_patch(circle)
                    # Add token
                    token = patches.Circle((x, y), 0.08, facecolor='black', edgecolor='black')
                    ax1.add_patch(token)
                elif label == 'p10':  # End place
                    circle = patches.Circle((x, y), 0.25, facecolor='white', edgecolor='black', linewidth=3)
                    ax1.add_patch(circle)
                else:
                    circle = patches.Circle((x, y), 0.25, facecolor='white', edgecolor='black', linewidth=2)
                    ax1.add_patch(circle)
                
                ax1.text(x, y-0.5, label, ha='center', va='top', fontsize=9)
            
            # Draw transitions (squares)
            executed_events = getattr(create_trace_visualization, 'executed_events', [])
            for x, y, label in petri_elements['transitions']:
                # Color based on execution status
                event_name = label.replace('_', '')
                if event_name in executed_events or f"e{label.split('_')[1]}" in executed_events:
                    color = 'lightgreen'
                else:
                    color = 'black'
                
                square = patches.Rectangle((x-0.15, y-0.15), 0.3, 0.3, 
                                         facecolor=color, edgecolor='black', linewidth=2)
                ax1.add_patch(square)
                ax1.text(x, y-0.55, label, ha='center', va='top', fontsize=9, fontweight='bold')
            
            # Define arcs based on the LaTeX figure structure
            arcs = [
                # Initial flow
                ((0.8, 3.5), (1.8, 3.5)),  # p0 to t1
                ((1.8, 3.5), (3.2, 5.5)),  # t1 to p1
                ((1.8, 3.5), (3.2, 1.5)),  # t1 to p2
                # Parallel branches
                ((3.2, 5.5), (4.6, 5.5)),  # p1 to t2
                ((3.2, 1.5), (4.6, 1.5)),  # p2 to t3
                ((4.6, 5.5), (6.0, 5.5)),  # t2 to p3
                ((4.6, 1.5), (6.0, 1.5)),  # t3 to p4
                # Merge
                ((6.0, 5.5), (7.25, 3.5)), # p3 to t4
                ((6.0, 1.5), (7.25, 3.5)), # p4 to t4
                ((7.25, 3.5), (8.5, 3.5)), # t4 to p5
                # Next split
                ((8.5, 3.5), (10.0, 4.2)), # p5 to t5
                ((8.5, 3.5), (10.0, 2.8)), # p5 to t6
                ((10.0, 4.2), (11.5, 3.5)), # t5 to p6
                ((10.0, 2.8), (11.5, 3.5)), # t6 to p6
                # Next transition
                ((11.5, 3.5), (12.75, 3.5)), # p6 to t7
                ((12.75, 3.5), (14.0, 5.5)), # t7 to p7
                ((12.75, 3.5), (14.0, 1.5)), # t7 to p9
                # Final branches
                ((14.0, 5.5), (15.25, 5.5)), # p7 to t8
                ((14.0, 1.5), (14.75, 1.5)), # p9 to t11
                ((14.0, 1.5), (16.75, 2.5)), # p9 to t10
                ((15.25, 5.5), (16.5, 5.5)), # t8 to p8
                ((14.75, 1.5), (16.75, 2.5)), # t11 to t10 (via place)
                # Final convergence
                ((16.5, 5.5), (16.0, 3.5)), # p8 to t9
                ((16.75, 2.5), (16.0, 3.5)), # t10 to t9
                ((16.0, 3.5), (17.5, 3.5))   # t9 to p10
            ]
            
            # Draw arcs
            for (x1, y1), (x2, y2) in arcs:
                ax1.annotate('', xy=(x2, y2), xytext=(x1, y1),
                            arrowprops=dict(arrowstyle='->', color='black', lw=1.5))
            
            ax1.set_xticks([])
            ax1.set_yticks([])
            ax1.grid(True, alpha=0.3)
            
            # Middle: Goal Model (full width)
            ax2.set_title("Goal Model Structure", fontsize=16, fontweight='bold', pad=20)
            layout = Layout(model)
            ax2.set_xlim(0, layout.max[0])
            ax2.set_ylim(0, layout.max[1])
            ax2.set_aspect('equal')
            
            fontsize=10
            
            positions = layout.positions
            
            # Draw goal model elements
            shapes = {}
            for element_id, (x, y) in positions.items():
                color = get_status_color_from_your_model(element_id)
                
                if model._get_element_type(element_id) == "Quality":
                    # Quality - cloud shape
                    cloud = FancyBboxPatch((x-0.6, y-0.4), 1.2, 0.8, 
                                         boxstyle="roundtooth, pad=0.6, tooth_size=0.5", 
                                         facecolor=color, edgecolor='black', linewidth=2)
                    ax2.add_patch(cloud)
                    shapes.update({element_id:cloud})
                    status_text = f"{element_id}\n{model._format_status(model.qualities[element_id])}"
                    ax2.text(x, y, status_text, ha='center', va='center', fontweight='bold', fontsize=fontsize,  zorder=10)
                elif model._get_element_type(element_id) == "Goal":
                    # Goal - ellipse
                    ellipse = patches.Ellipse((x, y), 1.0, 0.6, 
                                            facecolor=color, edgecolor='black', linewidth=2)
                    ax2.add_patch(ellipse)
                    shapes.update({element_id:ellipse})
                    status_text = f"{element_id}\n{model._format_status(model.goals[element_id])}"
                    ax2.text(x, y, status_text, ha='center', va='center', fontweight='bold', fontsize=fontsize)
                else:
                    # Task - hexagon
                    hexagon = patches.RegularPolygon((x, y), 6, radius=0.5, 
                                                  facecolor=color, edgecolor='black', linewidth=2)
                    ax2.add_patch(hexagon)
                    shapes.update({element_id:hexagon})
                    status_text = f"{element_id}\n{model._format_status(model.tasks[element_id])}"
                    ax2.text(x, y, status_text, ha='center', va='center', fontweight='bold', fontsize=fontsize)
                            
            # Draw links in goal model
            for parent, child, link_type, _ in model.links:
                shrink = 2
                if link_type == LinkType.MAKE:
                        arrow_color = 'green'
                        style = '->'
                elif link_type == LinkType.BREAK:
                        arrow_color = 'red'
                        style = '->'
                elif link_type == LinkType.AND:
                        arrow_color = 'purple'
                        style = '|-|,widthA=0,widthB=0.5'
                        shrink = 20
                elif link_type == LinkType.OR:
                        arrow_color = 'orange'
                        style = '->'
                else:
                        arrow_color = 'blue'
                        style = '->'
                        
                connector_arrow = patches.FancyArrowPatch(
                                        posA=positions[child],
                                        posB=positions[parent],
                                        patchA=shapes[child], 
                                        patchB=shapes[parent],
                                        arrowstyle=style,
                                        color=arrow_color,
                                        linewidth=4,
                                        shrinkB=shrink,
                                        mutation_scale=20
                                        )
                                        
                ax2.add_patch(connector_arrow)
            
            ax2.set_xticks([])
            ax2.set_yticks([])
            ax2.grid(True, alpha=0.3)
            
            # Bottom: Horizontal Mapping Table
            ax3.set_title("Process Transition to Goal Element Mapping", fontsize=14, fontweight='bold', pad=15)
            ax3.axis('off')  # Hide axes for table
            
            # Create horizontal table data (transposed)
            transitions = ['t_1', 't_2', 't_3', 't_4', 't_5', 't_6', 't_7', 't_8', 't_9', 't_10', 't_11']
            elements = ['ue', 'ue', 'ra', 'g', 'g', 'o', 'pt', 'pt', 'fs', 'fs', 'ap']
            
            # Create table with transitions as columns
            table_data = [
                transitions,  # First row: transition names
                elements      # Second row: corresponding elements
            ]
            
            # Create horizontal table
            table = ax3.table(cellText=table_data,
                             rowLabels=['Process Transition', 'Goal Element'],
                             cellLoc='center',
                             loc='center',
                             colWidths=[0.08] * len(transitions))  # Equal width columns
            
            # Style the table
            table.auto_set_font_size(False)
            table.set_fontsize(10)
            table.scale(1, 2)
            
            # Style row labels
            table[(0, -1)].set_facecolor('#4472C4')
            table[(0, -1)].set_text_props(weight='bold', color='white')
            table[(1, -1)].set_facecolor('#4472C4')
            table[(1, -1)].set_text_props(weight='bold', color='white')
            
            # Style data cells with alternating colors
            for i in range(len(transitions)):
                if i % 2 == 0:
                    table[(0, i)].set_facecolor('#F2F2F2')
                    table[(1, i)].set_facecolor('#F2F2F2')
                else:
                    table[(0, i)].set_facecolor('white')
                    table[(1, i)].set_facecolor('white')
            
            # Use subplots_adjust instead of tight_layout to avoid table compatibility issues
            plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05, hspace=0.35)
            plt.show()
            plt.close()
    
    return viz_output, update_dual_visualization

print("Trace and Evolution visualization functions ready!")
print("Functions created:")
print("- create_trace_visualization(): Horizontal timeline display")
print("- create_evolution_visualization(): Event-to-element mapping visualization")  
print("- create_dual_model_visualization(): Full-width models with horizontal mapping table")

## Layout - The Interactive Notebook

In [9]:
# Cell 8: Main Interface with New Layout
def display_interface():
    """Display the interface with trace timeline and dual model view"""
    
    # Header
    header = widgets.HTML(f"""
    <div style='text-align: center; margin-bottom: 20px;'>
        <h1 style='color: #2E86AB; margin-bottom: 10px;'> A unified view - Interactive High-Level Business Requirements Evaluation</h1>
        <p style='font-size: 14px; color: #666;'>Using your actual GoalModel class from: {implementation_path}</p>
    </div>
    """)
    
    # Legend - more compact
    legend = widgets.HTML("""
    <div style='background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin-bottom: 15px; font-size: 12px; border: 1px solid #ccc;'>
    <div style='font-weight: bold; margin-bottom: 8px; text-align: center;'>Conventions</div>
    <div><strong>Colors:</strong> 🤍 Unknown | 🟢 Satisfied/Fulfilled | 🔵 Executed Pending | 🔴 Denied</div>
    <div><strong>Shapes:</strong> ☁️ Quality | ⭕ Goal | ⬡ Task | ⬜ Process Transition</div>
    </div>
    """)
    
    # Controls section
    controls_section = widgets.VBox([
        widgets.HTML("<h3>Controls</h3>"),
        controls,
        status_out
    ])
    
    # Create trace and dual visualization
    trace_out_new, update_trace = create_trace_visualization()
    evolution_out, update_evolution = create_evolution_visualization()
    dual_viz_out, update_dual_viz = create_dual_model_visualization()
    
    # Modified execute function to update all displays
    def execute_event_with_trace(b):
        """Execute event and update all displays"""
        selected_event = controls.children[0].value  # Get dropdown value
        
        with status_out:
            clear_output(wait=True)
            print(f" Executing event: {selected_event}")
            
            # Add to trace
            create_trace_visualization.executed_events.append(selected_event)
            
            # Use your actual process_event method
            model.process_event(selected_event)
        
            print(f" Event {selected_event} processed!")
            
            # Show current quality status
            print("\n Current Status:")
            for quality_id, status in model.qualities.items():
                formatted_status = model._format_status(status)
                print(f" {quality_id}: {formatted_status}")
        
        # Update all displays
        update_trace()
        update_evolution()
        update_dual_viz()
    
    def reset_model_with_trace(b):
        """Reset model and clear trace"""
        global model
        model = create_model_from_your_code()
        create_trace_visualization.executed_events = []
        
        with status_out:
            clear_output(wait=True)
            print("🔄 Model and trace reset to initial state")
        
        update_trace()
        update_evolution()
        update_dual_viz()
    
    # Clear any existing event handlers to prevent duplicates
    controls.children[1]._click_handlers.callbacks.clear()  # Execute button
    controls.children[2]._click_handlers.callbacks.clear()  # Reset button
    
    # Connect new event handlers
    controls.children[1].on_click(execute_event_with_trace)  # Execute button
    controls.children[2].on_click(reset_model_with_trace)    # Reset button
    
    # Create main tab structure
    main_tab = widgets.Tab()
    
    # Main tab content with trace timeline, evolution view, and dual models
    main_content = widgets.VBox([
        controls_section,
        trace_out_new,
       # widgets.HTML("<h3>Evolution View</h3>"),
       # evolution_out,
        widgets.HTML("<h3>Model Views</h3>"),
        dual_viz_out
    ])
    
    main_tab.children = [main_content]
    main_tab.set_title(0, 'Main Interface')
    
    # ONLY call the initial visualization ONCE here
    update_trace()
    update_dual_viz()
    
    # Complete interface
    interface = widgets.VBox([
        header,
        legend,
        main_tab
    ])
    
    return interface

# Display the complete interface
interface = display_interface()
display(interface)

print("Interactive interface ready!")
print("This notebook uses your actual code files:")
print(f"   - {implementation_path}/enums.py")
print(f"   - {implementation_path}/goal_model.py")
print("Use the interface above to interact with your GoalModel!")

Interactive interface ready!
This notebook uses your actual code files:
   - c:\Users\jcavi\Documents\GitHub\Kogi-Python\Implementation/enums.py
   - c:\Users\jcavi\Documents\GitHub\Kogi-Python\Implementation/goal_model.py
Use the interface above to interact with your GoalModel!
